# Food Recommendations

Below is the output of a recommender system for healthy foods.  

In [1]:
# Imports and settings
import pandas as pd
from Recommender_system.Recommender import UserRatings, collabFilteringModel, getUserFavFoods, getMealRec, format_meal_rec

pd.options.display.max_colwidth = 100

In [2]:
# Load dataset
user_food_table = pd.read_csv("Recommender_system/Dataset.csv")

### Set User and Meal Kind

Here, set the user ID and kind of meal (`'breakfast'`, `'lunch'`, `'dinner'`, or `''` for all meals)

In [3]:
user_id = 445
meal_kind = 'lunch'

## 1. Summary of User's Meals

Below are the food items in the user's food log for this meal type

In [4]:
user_ratings = UserRatings(user_food_table, meal_kind)
user_food_list, meal_summary = user_ratings.mealSummary(user_id)
meal_summary

,list of foods
meal_id,
1124,"butternut squash, ravioli, sauce"
1158,"chicken breast, shrimp, rice, guacamole"
1181,"sweet potato, quinoa, brown rice, black beans, sweet potato, tomato, onion"
1196,"soup, chicken broth, carrots, onions, celery, pasta"
1203,"basmati rice, lamb, tomatoes, onions, peas, basmati rice"
1222,"chicken broth, corn, onions, potatoes, celery, milk"


## 2. Recommending food items

In [5]:
# Run and fit the food recommendation model
food_rec_model = collabFilteringModel(d=8, sigmasq=0.6, lambd=1, nUsers=user_ratings.nUsers, nFoods=user_ratings.nFoods)
food_rec_model.fit(user_ratings.ratings_train.values,seed_u=3,seed_v=5) #set the seed to get reproducible results

collabFilteringModel(d=8, lambd=1, nFoods=469, nUsers=116, sigmasq=0.6)

In [6]:
# Get favorite and least favorite foods
fav_foods = getUserFavFoods(user_id, user_ratings, food_rec_model)
fav_foods = fav_foods[~fav_foods.food.isin(user_food_list)]

In [7]:
top_fav_foods = fav_foods.iloc[0:10,].food
top_fav_foods = top_fav_foods.sample(5, random_state=123)

In [8]:
least_fav_foods = fav_foods.iloc[-10:-1,].food
least_fav_foods = least_fav_foods.sample(5, random_state=123)

Here are 10 food items - based on what the user has eaten before, please rate the extent to which you think the user would or would not like each food item on a scale from 1, the user would not like this food item at all, to 7, the user would like this food item a lot.

In [9]:
test_food_list = pd.DataFrame({'label': 'fav', 'food': top_fav_foods})
test_food_list = test_food_list.append(pd.DataFrame({'label': 'least_fav', 'food': least_fav_foods})).sample(frac=1).reset_index()
test_food_list['rating'] = '1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)'
test_food_list[['food','rating']]

,food,rating
0,sloppy joe,1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)
1,veggie straws,1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)
2,bun,1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)
3,flax,1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)
4,sun chips,1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)
5,potato chips,1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)
6,1% milk,1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)
7,bagel,1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)
8,hero,1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)
9,turkey breast,1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)


## 3. Recommending complete meals

In [21]:
association_rules = user_ratings.calculateAssociationRules(0.005)

In [22]:
seed_food = 'rye bread'

In [23]:
meal_rec = getMealRec(user_id, user_ratings=user_ratings, model=food_rec_model, seed_food=seed_food)
suggestion = meal_rec.iloc[0,:]

In [24]:
rec = format_meal_rec(suggestion, seed_food)
rec

'To go with 2 servings of rye bread, try 1 serving of almond milk and bacon bits'

How would you rate this recommendation on a scale from 1 to 7?

`1 - 2 - 3 - 4 - 5 - 6 - 7`

In [25]:
seed_food = 'chicken'
meal_rec = getMealRec(user_id, user_ratings=user_ratings, model=food_rec_model, seed_food=seed_food)
suggestion = meal_rec.iloc[0,:]
rec = format_meal_rec(suggestion, seed_food)
rec

'To go with 2 servings of chicken, try 1 serving of carrot, cucumber and tomato'

How would you rate this recommendation on a scale from 1 to 7?

`1 - 2 - 3 - 4 - 5 - 6 - 7`